# Move onto Bigger and More Entangled Things: Programming with multiple qubits


Execute the cell below to load all the necessary packages for this lab.

In [ ]:
!pip install cudaq
!pip install matplotlib
!pip install qutip

In [ ]:
import cudaq
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output


## Program with more than one qubit


In this section, we'll create two quantum programs to illustrate the concepts of global phase and entanglement.  Additionally, these examples will allow us to demonstrate alternative ways of writing and combining multiple kernels &mdash; a useful technique to have on hand as our quantum algorithms grow in the number of qubits and the number of gate operations.  Both the number of qubits and the number of gate operations are limiting factors in the kind of quantum algorithms that can be run on current quantum hardware or simulated on GPUs.  In fact, we'll quickly see examples of quantum algorithms that we can program, but cannot implement with today's existing technology.



### Notation for a 2-qubit state
$\renewcommand{\ket}[1]{|#1\rangle}$
In Lab 1, we saw that the quantum state of a single qubit could be written as linear combinations of the states $|0\rangle$ and $|1\rangle$.  The states $|0\rangle$ and $|1\rangle$ are often referred to as the computational basis states of a single qubit.  When we have 2 qubits, we will need $2^2 = 4$ basis states to describe all possible quantum states of 2 qubits because each qubit could be measured as either a $0$ or a $1$.  The computational basis states used to describe a state of 2 qubits are
$$\ket{00}, \ket{01}, \ket{10},  \text{ and }\ket{11}.$$

What does this notation mean? The expression $\ket{10}$ represents the state of a system of 2 qubits, (e.g., $q_0$ and $q_1$), where qubit $q_0$ is in the state $\ket{1}$ and $q_1$ is in the state $\ket{0}$.  In other words, the left most term (the $1$ in $\ket{10}$) corresponds to qubit $q_0$'s state and the rightmost term will correspond to the state of the 2nd qubit.  Since there is no agreed upon standard ordering of qubits, this notational choice may differ from other quantum programming languages that you might run across.

The general form of a quantum state of 2 qubits, is a linear combination (or a superposition) of the computational basis states:

$$\ket{\psi} = \alpha_{00}\ket{00}+\alpha_{10}\ket{10}+\alpha_{01}\ket{01}+\alpha_{11}\ket{11},$$
where each coefficient (also called a **probability amplitude**) $\alpha_{ij}$ is a complex number and the sum of the probabilities corresponding to these coefficients is 1, that is: $$\sum_{i,j\in\{0,1\}} |\alpha_{ij}|^2 =1.$$  

Sometimes it is useful, albeit often less compact, to write the state $\ket{\psi}$ as a vector of probability amplitudes:
$$\ket{\psi} = \alpha_{00}\ket{00}+\alpha_{10}\ket{10}+\alpha_{01}\ket{01}+\alpha_{11}\ket{11}
 = \begin{pmatrix}\alpha_{00} \\ \alpha_{01} \\ \alpha_{10} \\ \alpha_{11}\end{pmatrix}$$

> **Note:** The ordering listed in the sum of the computational basis states $(00, 10, 01, 11)$ differs from the ordering in the vertical vector. We introduce both of these conventions because the ordering  $(00, 10, 01, 11)$ is used when we execute the `get_state` function, while the ordering in the vertical vector $(00, 01, 10, 11)$ is used in matrix and tensor algebra computations.

In the literature, the term **statevector** refers to a quantum state $\ket{\psi}$, but can also refer to the list of probability amplitudes of a quantum state $\ket{\psi}$.  



### Write and interpret a 2-qubit quantum program

In this section, we'll write a program that builds upon the Bit-Flip and Hello World (minus state) programs from Lab 1.  In particular, our goal is to create one program with 2 qubits.  The first qubit will be initialized in the zero state, and we'll flip it to the one state.  The second qubit will be initialized in the minus state, and just for fun we'll apply the bit flip gate `x` to it too. The diagram represents the circuit for this program.

![quantum circuit diagram with 2 qubits](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/2-qubit-circuit.png?raw=1)

In our example, qubit $q_0$, which starts in the $\ket{0}$ state undergoes a bit flip.  So we would expect, as we saw in Lab 1, that when the kernel is sampled in a noiseless setting, $q_0$ will be measured  $100\%$ of the time as a $\ket{1}$.   On the other hand, because qubit $q_1$ will be in a state of equal superposition, it will collapse to the $\ket{0}$ state approximately half of the time and will collapse to $\ket{1}$ the other times. Putting the information about the state of $q_0$ and $q_1$ together, we expect a $50\%$ probability of sampling a $\ket{10}$ and  $50\%$ probability of sampling a $\ket{11}$ when running the program depicted above.

The statevector of the 2-qubit system after the gates are applied, but  prior to measurement should be $$\ket{\psi} = -\frac{1}{\sqrt{2}}\ket{10} +\frac{1}{\sqrt{2}}\ket{11} = \begin{pmatrix}0 \\ 0 \\ -\frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}} \end{pmatrix}$$

To understand the appearance of the minus sign in the equation above, refer to the optional section on phase later in this notebook.  For now, just notice that by squaring the absolute values of the amplitudes, we see a $50\%$ probability of sampling a $\ket{10}$ and  $50\%$ probability of sampling a $\ket{11}$.

We have several options for writing the CUDA-Q code to carry out the instructions above.  In the next subsections, we'll take a look at two options so that you can see some of the functionality of `cudaq.kernel`. In particular, we'll demonstrate
* how to pass individual qubits or a register of qubits to a kernel and
* how to use kernels as subkernels (subroutines).



#### All-in-one kernel

We'll first code the quantum circuit like we did in Lab 1, by writing all the instructions sequentially in one kernel.

In [ ]:
# Writing the all-in-one kernel

@cudaq.kernel
def all_in_one():
    """Creates a kernel with 2 qubits. The first qubit is initialized
    in the zero state, the second qubit is initialized in the minus state.
    An x gate is then applied to each qubit.
    """
    # Allocate 2 qubits (indexed by 0 and 1)
    qvector = cudaq.qvector(2) # By default these qubits are the zero state

    # Initialize qubit indexed with 1 as the minus state
    x(qvector[1])
    h(qvector[1])

    # Manipulate the quantum states by applying the bit flip gate (the `x` gate) to each qubit
    x(qvector) # We can apply a single-qubit gate to each qubit in a list of qubits

print(cudaq.draw(all_in_one))



Let's check that this kernel generates our target state $\ket{\psi} = -\frac{1}{\sqrt{2}}\ket{10} +\frac{1}{\sqrt{2}}\ket{11}$ by using the `get_state` command.  We'll also use the `get_state.amplitudes` option to extract out the coefficients of the computational basis states $\ket{10}$ and $\ket{11}$ that are of interest to us.

In [ ]:
# Compute the state of the system prior to measurement
all_in_one_state = cudaq.get_state(all_in_one)

# Return the amplitudes of |10> and |11>
all_in_one_amplitudes = all_in_one_state.amplitudes([[1,0], [1,1]])

# Print
precision = 4
print('All-in-one statevector array of coefficients:', np.round(np.array(all_in_one_state), precision))
print('All-in-one statevector: {} |10> + {} |11>'.format(np.round(all_in_one_amplitudes[0],precision), np.round(all_in_one_amplitudes[1],precision)))


> **Quick Tip:**

To remember the ordering of the coefficients in the statevector generated by the `get_state` command, it helps to recall the convention for associating numbers with their binary representations that places the least significant bit on the left.  For example, in a 2-bit system, we have the following translation between integers and bits:
$$\begin{matrix} \text{Integer} & \text{Binary representation}\\
& \text{least significant bit on left}\\
0 =\color{red}{0}*2^0+\color{blue}{0}*2^1 & \color{red}{0}\color{blue}{0} \\
1 = \color{red}{1}*2^0 + \color{blue}{0} *2^1 & \color{red}{1}\color{blue}{0}\\
2 = \color{red}{0}*2^0 + \color{blue}{1}*2^1 & \color{red}{0}\color{blue}{1} \\
3 = \color{red}{1}*2^0 + \color{blue}{1}*2^1 & \color{red}{1}\color{blue}{1} \end{matrix}
$$

> The `get_state` output generates a list of the coefficients in increasing order: $\ket{00}, \ket{10}, \ket{01}, \ket{11}$, corresponding to $0,1,2,3.$ To conveniently identify the coefficient for a given basis state, use the `get_state().amplitude()` command. For example, to extract the coefficient for the basis state $\ket{10}$, use `get_state(my_kernel).amplitude([1,0])`.  To retrieve the amplitudes of more than one state at a time, use the `amplitudes` option applied to a list of basis states, as demonstrated in the code block above.


#### Kernels as subroutines

In this section, we will create the same quantum circuit as above, but this time we'll use kernels as subroutines. We'll create three separate kernels:

* `minus`, a kernel for the subroutine that initializes a given qubit as the minus state
* `xgate`, a kernel that applies the `x` gate to a list of qubits
* `nested_quantum_program`, a kernel to allocate the qubits and call the `minus` and `xgate` subroutines

Let's see how this is done. Take note of how we can pass qubits and lists of qubits to the kernel functions.

In [ ]:
# Defining three kernels to generate the example circuit

# Defining the subroutine, minus, as a kernel
@cudaq.kernel
def minus(qubit: cudaq.qubit):
    """When applied to a qubit in the zero state, generates the minus state
    Parameters
    ----------
    qubit : cudaq.qubit
        qubit upon which the kernel instructions will be applied
    """
    x(qubit)
    h(qubit)

# Defining the subroutine, xgate, as a kernel
@cudaq.kernel
def xgate(qubits: cudaq.qvector):
    """Applies an x-gate to each qubit in qubits
    Parameters
    ----------
    qubits : cudaq.qvector
        list of qubits upon which the kernel instructions will be applied
    """
    x(qubits)

# Allocating qubits and calling the subroutines defined above
@cudaq.kernel
def nested_quantum_program(num_qubits: int):
    """Creates a kernel with num_qubits-many qubits. The first qubit is initialized
    in the zero state, the second qubit is initialized in the minus state.
    An x gate is then applied to each qubit. Here num_qubits is at least 2.
    Parameters
    ----------
    num_qubits : int
        Number of qubits to be allocated
    """

    # Allocate num_qubits qubits indexed from 0 to num_qubits-1
    qvector = cudaq.qvector(num_qubits) # By default these qubits are in the zero state

    # Initialize the qubit indexed with 1 as the minus state
    # by calling the minus kernel acting on qvector[1]
    minus(qvector[1])

    # Manipulate the quantum states by applying the bit flip gate (the `x` gate) to each qubit
    # by calling the xgate kernel applied to a list of qubits
    xgate(qvector)


num_qubits = 2 # num_qubits should be an integer > 1

print(cudaq.draw(nested_quantum_program,num_qubits))



### Exercise 1
Edit the <code>##FIX_ME##</code> portion of the code block below that uses the `get_state.amplitudes` to verify that the <code>nested_quantum_program</code> generates the desired state $\ket{\psi} = -\frac{1}{\sqrt{2}}\ket{10}+\frac{1}{\sqrt{2}}\ket{11}$ when <code>num_qubits</code> is set to $2$.  


In [ ]:
# EXERCISE 1
num_qubits = 2

amplitudes = ##FIX_ME##
print('|Psi>= {} |10> + {} |11>'.format(np.round(amplitudes[0],precision), np.round(amplitudes[1],precision)))

Now let's use the `sample` command to compare the two quantum programs. Notice the difference between the arguments in the `sample` command applied to the kernel `all_in_one` versus the
`sample` command applied to the kernel `nested_quantum_program` which takes as an argument `num_qubits`.

In [ ]:
# Compare the quasi-probability distributions of each kernel upon sampling
shots = 1000

# Sample the kernels
counts_all_in_one = cudaq.sample(all_in_one, shots_count=shots)
counts_nested = cudaq.sample(nested_quantum_program, num_qubits, shots_count=shots)


# Print
print('All-in-one sampling results:', counts_all_in_one)
print('Nested results:', counts_nested)

> **Quick Tip:**
The `sample` command runs the kernel and takes a measurement `shots_count`-many times. The result of each kernel execution and measurement is a classical bitstring with the $i^{th}$ bit representing the measurement of the $i^{th}$ qubit. The result of the `sample` command is a `SampleResult` dictionary of the distribution of the bitstrings.  To create a more familiar `dict` Python type from your variable `myresult` of type `SampleResult`, use `asDict = {k:v for k,v in myresult.items()}`.


Before going on, let's interpret the output of the sample command.
Based on our prediction the output `{ 11:481 10:519 }` indicates that the first digit in the bitstrings `11` and `10` corresponds to qubit $q_0$, as it is `1` $100\%$ of the time, and the second digit corresponds to qubit $q_1$, which gets measured about half the time as a `0` and the other half of the time as a `1`.  



> **CUDA-Q Quick Tip:** The code blocks in this section demonstrate the ability for kernels to accept variables and to call one another.  It'll be useful to note:

> * CUDA-Q kernels can only accept variables of certain types.  In particular, you can pass an individual qubit (`cudaq:qubit`), a list of qubits (`cudaq:qview`), integers (`int`), floats (`float`), complex values (`complex`), and lists of integers (`list[int]`),floats (`list[float]`), and complex values (`list[complex]`).  The full list of allowable types can be found [here](https://nvidia.github.io/cuda-quantum/latest/specification/cudaq/kernels.html).
> * When sampling a kernel that accepts variables, the values for these variables, should appear in the `sample` function as a list following the kernel name, as seen in the example above where our kernel `nested_quantum_kernel` accepted an integer value `num_qubits`: `cudaq.sample(nested_quantum_program, num_qubits, shots_count=shots)`

### Exercise 2
Write a quantum program, using subkernels, to initialize three qubits in the zero state and place each qubit in the state $\ket{+}$ if it has even index and qubits with odd index should be placed in the $\ket{-}$ state (i.e., place $q_0$ and $q_2$ in the plus state and $q_1$ in the minus state.)  Use the `get_state` command to verify that your quantum program generates the desired quantum state. Print out the statevector to confirm. As a bonus, try making your program generic enough that it would work for any number of qubits.

For example, the state of a 3 qubit system prior to measurement should be:

\begin{aligned}\ket{\psi} &= \ket{+-+} \\ &= (\frac{1}{\sqrt{2}}(\ket{0}+\ket{1}))\otimes(\frac{1}{\sqrt{2}}(\ket{0}-\ket{1}))\otimes(\frac{1}{\sqrt{2}}(\ket{0}+\ket{1}) )\\ &= \frac{1}{\sqrt{8}}(\ket{000}+\ket{100}-\ket{010}-\ket{110}+\ket{101}+\ket{001}-\ket{011}-\ket{111}).\end{aligned}

The $\otimes$ notation above represents a tensor product. For now, you can treat this as a multiplication that satisfies standard multiplication properties such as transitivity and associativity. We will not be using this notation beyond this example. If you'd like a formal introduction to tensor products, check out <a href="https://www.thomaswong.net/introduction-to-classical-and-quantum-computing-1e4p.pdf" style="text-decoration: underline;">section 4.2.1 of this resource</a>.
    
> **CUDA-Q Quick Tip:** Inside a `cudaq.kernel` you can use python commands like `for` loops and `if` statements.  


In [ ]:
# EXERCISE 2

num_qubits = 3
shots = 1000

@cudaq.kernel
def alternating_signs(qubit_count: int):
    ###FIX_ME###

# Draw the circuit
print(cudaq.draw(alternating_signs,num_qubits))

# Verify state
# Compute the state of the system prior to measurement
state = ###FIX_ME###

# Print
precision = 4
print('Statevector array of coefficients:', np.round(np.array(state), precision))

#### Phase

Phase is a characteristic of quantum states that we have not yet discussed. Estimating the phase of a given quantum state is an important step in many quantum algorithms including Shor's algorithm.  Additionally, phase helps explain the phenomena of **interference** which is key in algorithms such as Grover's.    

Every state has a phase factor.  The **phase factor** of a state can be deduced from the term $e^{i\varphi}$ in the standard representation of a quantum state: $$
\ket{\psi} = \cos(\frac{\theta}{2})\ket{0}+\sin(\frac{\theta}{2})e^{i\varphi}\ket{1}. $$

We are often interested when two states have different phases.  This can happen in one of two ways: differing by a global phase or by a relative phase.  We've seen examples of both of these already in Lab 1 and in the previous examples in this lab.  

Let's take a deeper look at qubit $q_1$ in the `nested_quantum_program` example from the previous section.  We can follow the state of $q_1$ as it changes throughout the kernel execution. We begin in the state $\ket{0}$ which is flipped to $\ket{1}$. Then, an application of the $H$ gate takes us to $\ket{-} = \frac{1}{\sqrt{2}}\ket{0}-\frac{1}{\sqrt{2}}\ket{1}$.  This is followed by an application of the bit flip operator $X=\begin{pmatrix}0 & 1 \\ 1 & 0\end{pmatrix}$.  Carrying out this matrix multiplication we get that the state of $q_1$ prior to the measurement is:
$$ X\ket{-}=\begin{pmatrix}0 & 1 \\ 1 & 0\end{pmatrix}\begin{pmatrix} \frac{1}{\sqrt{2}} \\ -\frac{1}{\sqrt{2}}\end{pmatrix}
= \begin{pmatrix} -\frac{1}{\sqrt{2}} \\ \frac{1}{\sqrt{2}}\end{pmatrix} = -\ket{-}.
$$

When we measure $ -\ket{-}$ we get a similar quasi-probability distribution that we would get if we measured  $\ket{-}$.  We saw this behavior in Lab 1, where $\ket{+}$ and $\ket{-}$ had indistinguishable quasi-probability distributions upon measurement with `mz`.  However, if we measured the states $\ket{+}$ and $\ket{-}$ with `mx` we could distinguish the two states through their quasi-probability distributions.  Unlike in Lab 1, there is no measurement that we can apply to $\ket{-}$ and $-\ket{-}$ to distinguish them.

The state $\ket{-}$ and $-\ket{-}$ only differ by a scalar multiple of $-1$.  When states only differ by a factor of $-1$, they are said to differ by a **global phase**.  And these states cannot be distinguished from one another through any measurements, so they are often treated as being equivalent, or interchangeable.

This is in contrast to the states $$\ket{+} =  \frac{1}{\sqrt{2}}\ket{0}+\frac{1}{\sqrt{2}}\ket{1}=\frac{1}{\sqrt{2}}\ket{0}+e^{i*0}\frac{1}{\sqrt{2}}\ket{1}$$ and $$\ket{-}=  \frac{1}{\sqrt{2}}\ket{0}-\frac{1}{\sqrt{2}}\ket{1}=\frac{1}{\sqrt{2}}\ket{0}+e^{i\pi}\frac{1}{\sqrt{2}}\ket{1},$$ which have equal amplitudes but differ by a **relative phase**, in this case $\pi$.  

In general the **relative phase** difference between two states
$$
\ket{\psi_1} = \cos(\frac{\theta}{2})\ket{0}+\sin(\frac{\theta}{2})e^{i\varphi_1}\ket{1} $$
and  
$$
\ket{\psi_2} = \cos(\frac{\theta}{2})\ket{0}+\sin(\frac{\theta}{2})e^{i\varphi_2}\ket{1} $$
is defined to as $\varphi := |\varphi_1-\varphi_2|$

**Question:** What is the relative phase difference between $\ket{+}$ and $\ket{i}$?



### Write a "Hello, Entangled World!" program

So far, all of the quantum operations (e.g. `x`, `h`) that we programmed acted only on an individual qubit.  In this section, we'll write a program using multi-qubit gates.  Let's start with the example of the controlled-not gate, often abbreviated the `CNOT` gate. This is a gate that operates on 2 or more qubits. One of the qubits is considered the "target", and the remaining qubits are the "control".  For instance `CNOT([q0,q1],q2)` would represent a gate with control qubits in the list `[q0,q1]` and target qubit `q2`.  

In a circuit diagram we depict the `CNOT` gate as a line connecting the control and target qubits with a $\bigoplus$ located at the target qubit and a solid circle on control qubits. For instance, in the circuit diagram below, we see a `CNOT(q0, q1)` and a `CNOT([q1,q2],q0)` gate.

![circuit](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/cnots.png?raw=1)

The rough idea is that if the "control" qubits are all in the state $\ket{1}$ then an application of the `CNOT` gate will apply a bitflip (i.e. an `x` gate) to the target qubit.  If on the other hand, at least one of the "control" qubits is in the $\ket{0}$ state the `CNOT` gate has no effect.

To illustrate this, let's code up a few examples.

> **Quick Tip:**
The syntax for the CNOT gate is `x.ctrl(control_qubit, target_qubit)` if we have only one control qubit.  When we have multiple control qubits, we send them as a list to the `x.ctrl` function: `x.ctrl(control_qubits_list, target_qubit)`.

In [ ]:
# CNOT applied to the states |00>, |01>, |10>, |11>
print('In this example CNOT is applied with the control qubit q0 and the target q1.')

@cudaq.kernel
def apply_cnot(control: cudaq.qubit, target: cudaq.qubit):
    """Apply an CNOT gate to the control and target qubits"""
    x.ctrl(control, target)

@cudaq.kernel
def initialize(qubits: cudaq.qview, state: list[int]):
    """Kernel to initialize the state indicated by the given list
    0: |0>, 1: |1>"""
    for idx in range(len(state)):
        if state[idx] == 1:
            x(qubits[idx])

@cudaq.kernel
def CNOT_example(state: list[int]):
    """Apply CNOT to the state given by the state numbers"""
    qubits = cudaq.qvector(len(state))
    # Initialize state
    initialize(qubits, state)
    # Apply CNOT to the first two bits
    if len(state) > 1:
        apply_cnot(qubits[0], qubits[1])

# Test computational basis states
for i in range(4):
    bits = format(i,'b')
    bit_list = [int(bit) for bit in bits.zfill(2)]
    result = cudaq.sample(CNOT_example, bit_list).most_probable()
    print(f'CNOT applied to the state |{bits.zfill(2)}> results in |{result}>')



While the previous examples showed CNOT operations on basis states (|0⟩ and |1⟩), the interactive tool below lets you explore more interesting quantum behaviors. Try selecting superposition states (|+⟩ or |-⟩) for either or both qubits to see how the CNOT gate functions. You'll discover some surprising quantum behaviors that have no classical counterpart!  

 <iframe src="https://nvidia.github.io/cuda-q-academic/quick-start-to-quantum/interactive_widget/cnot-visualization.html" width="800" height="600"></iframe>

> **Note:** If the widget does not appear above, you can access it directly using [this link](https://nvidia.github.io/cuda-q-academic/quick-start-to-quantum/interactive_widget/cnot-visualization.html).


### Exercise 3a
Edit the code block below to see the effect of the gate sequence in the diagram below on the qubits if $q_0$ was initialized as $\ket{+}$ and $q_1$ was initialized as $\ket{1}$
        <br><br>
![image of a circuit with 3 alternating cnots and q_0 initialized as |+> and q_1 initialized as |1>](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/exercise3a.png?raw=1)


In [ ]:

# EXERCISE 3 Part a
@cudaq.kernel
def alternating_cnots(qubit0: cudaq.qubit, qubit1: cudaq.qubit):
    """Apply a sequence of 3 CNOTs with alternating controls and targets on the 2 qubits given as arguments"""
    # We will see later that it doesn't matter which qubit you start out with as the first control, as long as
    # you alternate the control and target qubits with each of the 3 applications of the cnot gate

    # Edit code below this line

    # Edit code above this line


@cudaq.kernel
def three_alternating_cnots():
    """Kernel for the circuit drawn above"""

    # Allocate qubits
    q = cudaq.qvector(2)

    # Initialize qubits q0 and q1 in the plus and one states, respectively
    h(q[0])
    x(q[1])

    # Apply alternating CNOTs
    alternating_cnots(q[0], q[1])



results = cudaq.sample(three_alternating_cnots)
print('The distribution of states after sampling is: {}'.format(results))


Did you notice that the `alternating_cnots` kernel had the effect of swapping the states of $q_0$ and $q_1$?  This is a useful operation, so there is a dedicated notation for it.  We refer to this gate sequence as the `SWAP` gate, and abbreviate the gate sequence as a line with 2 Xs on the endpoints connecting the qubits:  


![image of a circuit with a swap gate and one of three alternating cnots](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/SWAP.png?raw=1)

The CUDA-Q command for a SWAP gate applied to qubits $q_0$ and $q_1$ is `swap(q_0, q_1)`. Let's use CUDA-Q to check that the `swap` gate has the same effect on two qubits as the `alternating_cnots` kernel.

In [ ]:
# Verify that the SWAP and alternating_cnots have the same effect on the state |+1>

@cudaq.kernel
def applying_swap():
    """Kernel for the circuit drawn above"""

    # Allocate qubits
    q = cudaq.qvector(2)

    # Initialize qubits q0 and q1 in the plus and one states, respectively
    h(q[0])
    x(q[1])

    # Apply alternating CNOTs
    swap(q[0], q[1])

results = cudaq.sample(three_alternating_cnots)
print('The distribution of states after sampling is: {}'.format(results))

Now, let's explore another example that will showcase one of the powerful computations achievable with quantum gates.


### Exercise 3b
Edit the ##FIX_ME## phrase in the code block below to apply the `x.ctrl` gate to the state $\ket{+0}$.


In [ ]:
# EXERCISE 3 Part b

@cudaq.kernel
def apply_cnot(control: cudaq.qubit, target: cudaq.qubit):
    """Apply an CNOT gate to the control and target qubits"""
    x.ctrl(control, target)

@cudaq.kernel
def initialize_plus_zero(qubits: cudaq.qview):
    """Kernel to initialize the state indicated by the given list of bits"""
    # Place qubits[0] in the plus state
    ##FIX_ME##

@cudaq.kernel
def CNOT_exercise():
    """Apply CNOT to |+0> with control q0 and target q1"""
    qubits =cudaq.qvector(2)
    # Initialize state
    initialize_plus_zero(qubits)
    # Apply CNOT to the first two bits
    apply_cnot(qubits[0], qubits[1])

results = cudaq.sample(CNOT_exercise)
print('CNOT applied to the state |+0> results in the distribution of states: {}'.format(results))


To understand the output of the exercise above, let's use the ket notation.  First we should point out that one characteristic of quantum gates that we've not mentioned explicitly is linearity.  For example, if we have a gate $U$ applied to the state $\ket{\psi}=\alpha\ket{0}+\beta\ket{1}$, we can understand the action of $U$ on $\ket{\psi}$ by knowing how $U$ affects $\ket{0}$ and $\ket{1}$.  In particular, we can distribute $U$ across the terms of $\ket{\psi}$:
$$ U\ket{\psi} = U(\alpha\ket{0}+\beta\ket{1}) =  U(\alpha\ket{0}) +  U(\beta\ket{1})=\alpha U\ket{0} + \beta U\ket{1}.$$
This generalizes beyond one-qubit gates to multi-qubit gates.

Therefore, when we apply the `CNOT` to $\ket{+0} = \frac{1}{\sqrt{2}}\ket{00}+ \frac{1}{\sqrt{2}}\ket{10}$ we get

$$CNOT\ket{+0} = CNOT(\frac{1}{\sqrt{2}}\ket{00}+ \frac{1}{\sqrt{2}}\ket{10})=\frac{1}{\sqrt{2}}CNOT\ket{00} + \frac{1}{\sqrt{2}}CNOT\ket{10} = \frac{1}{\sqrt{2}}\ket{00} +  \frac{1}{\sqrt{2}}\ket{11}.$$

### Multi-qubit gates as matrices

To understand why gate operations are linear, it helps to represent a multi-qubit gate as a matrix, just as we have done in Lab 1 for single qubit gates.
For example the matrix for the CNOT gate using the convention of CUDA-Q ordering is:
$$CNOT = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix}.
$$

Using the fact that  $\ket{+0} = \frac{1}{\sqrt{2}}\ket{00}+ \frac{1}{\sqrt{2}}\ket{10}$, we can verify our work from the previous section:

$$ CNOT\ket{+0} = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix}(\frac{1}{\sqrt{2}}\ket{00}+ \frac{1}{\sqrt{2}}\ket{10})
=  \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix}
\begin{pmatrix} \frac{1}{\sqrt{2}}  \\ 0 \\ \frac{1}{\sqrt{2}} \\ 0\end{pmatrix} =
\begin{pmatrix} \frac{1}{\sqrt{2}} \\ 0 \\ 0 \\ \frac{1}{\sqrt{2}}  \end{pmatrix} =
 \frac{1}{\sqrt{2}}\ket{00} +  \frac{1}{\sqrt{2}}\ket{11}
$$

The state $\frac{1}{\sqrt{2}}\ket{00} +  \frac{1}{\sqrt{2}}\ket{11}$ is an example of entanglement.  Along with superposition, this property helps to distinguish quantum algorithms from classical.  Let's take a look at this property next.

> **FAQ:** If quantum states can be represented as vectors, and quantum gates can be realized through matrix operations, then why do we need quantum computers if we can theoretically simulate any quantum circuit through matrix multiplication classically?

> **Answer:**  One thing to note is that as the number of qubits increases, the number of possible states increases exponentially and the size of the matrix representing the multi-qubit gate also increases exponentially.  For instance, a 3-qubit system (like the Exercise 4 in the next section) can be in one of $2^3 = 8$ states, and the size of the matrix representing a 3-qubit gate is $8\times 8$.  This suggests that straightforward attempts at using matrix multiplication to simulate quantum computation on CPUs fail for circuits with only 30 or so qubits.  With CUDA-Q's platform and GPU-accelerated built-in simulators, it is possible to simulate larger systems. That said, there may one day be a fault tolerant quantum computer that can carry out computations too big for even the largest classical supercomputers.




### Entanglement

Entanglement of two (or more) qubits is akin to the concept of dependence in linear algebra.  That is to say that two qubits are entangled if the state of one of the qubits depends on the other.  Formally the definition of entanglement relies on tensor products, which are beyond the scope of this Quick Start series of notebooks.  You can read more about tensor products and the precise definition of entanglement [here](http://mermin.lassp.cornell.edu/qcomp/CS483.html) or [here](https://qubit.guide).  For now, we'll illustrate entanglement through the example of the 2-qubit state
$$ \ket{\psi} =\frac{1}{\sqrt{2}}\ket{00} +  \frac{1}{\sqrt{2}}\ket{11}.$$

The state $\ket{\psi}$ has equal probability of being measured in the state $\ket{00}$ or $\ket{11}$.  Let's look at what happens if we measure the first qubit.

* If our system of two qubits is in the state $\ket{\psi}$ and we measure qubit 0 to be $0$, then the state of the system collapses to $\ket{00}$.  In this case, without taking a measurement on qubit 1, we already know its state (it's $\ket{0}$).  

![image of a circuit for the Bell State with the first qubit measured to be a 0](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/bell0.png?raw=1)


* Similarly if the state of our system is $\ket{\psi}$ and qubit 0 is measured to be 1, then  the state of the system collapses to $\ket{11}$.  Although prior to any measurement on qubit 0, qubit 1 had a $50\%$ probability of being in state $\ket{0}$ or $\ket{1}$, after measuring qubit 0 to be 1, we know that qubit 1 must be in the state $\ket{1}$.

![image of a circuit for the Bell State with the first qubit measured to be a 1](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/bell1.png?raw=1)


So we see that even though prior to measurement, both qubits 0 and 1 have equal probabilities of being measured a 0 or 1, once we measure one of these qubits, the outcome of the other qubit has been determined and depends on the state of the qubit that was measured.  In other words,  qubit 0 and qubit 1 are **entangled**.



### Exercise 4 

Edit the code block below to create a quantum program with 3 qubits to:
* Initialize the state $\ket{\psi} = \ket{+-+} = \frac{1}{\sqrt{8}}(\ket{000}+\ket{100}-\ket{010}-\ket{110}+\ket{101}+\ket{001}-\ket{011}-\ket{111})$ by editing the <code>##FIX_ME##</code>
* Apply a <code>x.ctrl</code> gate with control qubit $q_0$ and target $q_1$. Then apply a Hadamard gate to $q_1$.
* Sample the result


In [ ]:
# EXERCISE 4
num_qubits = 3

@cudaq.kernel
def initial_state(qubits : ##FIX_ME##):
    for index in range(len(qubits)):
        if index % 2 !=0:
            x(qubits[index])
    h(qubits)

@cudaq.kernel
def interference(qubit_count: int):
    qvector = cudaq.qvector(qubit_count)

    initial_state(qvector) # Initialize the state

    # Apply x.ctrl with control q_0 and target q_1. Then apply a Hadamard gate to q_1.
    # Edit the code below this line


    # Edit the code above this line

results = cudaq.sample(interference, num_qubits, shots_count = 1000)
print(results)

**Question**: What do you notice about the probability amplitudes in Exercise 4?  Pay specific attention to the probability amplitude for
$\ket{110}$, $\ket{011}$, $\ket{111}$, and $\ket{010}$ compared to those of the other basis states.

The changing of the probability amplitudes in this exercise is an example of interference, which we'll discuss next.

### Interference

In a quantum program, we apply gates to change quantum state from one state to another.  In effect, what we're doing is changing the probability amplitudes associated with each of the basis states.  This process is called **interference**.  Interference is central in almost every quantum algorithm.

We can have both constructive and destructive interference.  Like the classical interference of waves from raindrops on the surface of a puddle, constructive interference occurs when the probability amplitude of basis state is increased, and destructive interference occurs when the probability of a basis state decreases due to the action of quantum gates.  The previous example demonstrates both constructive and deconstructive interference. The circuit is initialized in a state having equal magnitude probability amplitudes, but after applying the CNOT gate and Hadamard gate, the probabilities of measuring $\ket{110}$, $\ket{011}$, $\ket{111}$, and $\ket{010}$ have each been amplified, while the probability amplitudes associated with the other basis states have been reduced to $0$.

We will see another example of interference in Lab 3 when explore quantum walks.



## Other multi-qubit gates

There are many more quantum gates than the handful that we've explicitly used so far.  In the previous lab, we mentioned that any rotation of the Bloch sphere corresponds to a quantum gate on one qubit. So in theory, there are an infinite number of possible quantum gates: one for each of the possible rotations of the sphere.  In this section, we'll first detail what these rotations gates look like both as matrices and as CUDA-Q operations. Then  we'll use these rotations gates to define a larger class of multi-qubit controlled gates.


### Revisiting single-qubit rotation gates

We've already seen the `x`-gate that rotates a state around the x-axis of the Bloch sphere by 180 degrees and the Hadamard gate which rotates the Bloch sphere 180 degrees about a different axis causing $\ket{0}$ to move to $\ket{+}$ and $\ket{1}$ to rotate to $\ket{-}$, and vice versa.  But how do we implement gates for other angles of rotation, or different axis of rotation?  One easy answer is to use some of the gate operations built into CUDA-Q.  These include the `y`-gate for 180 degree rotation about the $y$-axis and the `z`-gate for 180 degree rotation about the $z$ axis.  The `t` and `s` gates correspond to other fixed angle rotations about the $z$ axis.  


### Exercise  5:
    
 Using the visualization tool, experiment with applying the <code>t</code> and <code>s</code> gates to various states in the code block below to deduce the rotation angle for the <code>t</code> and <code>s</code> gates and to predict the relationship between the <code>t</code> and <code>s</code> gates. Follow [this link](https://nvidia.github.io/cuda-q-academic/quantum-applications-to-finance/images/gate-widget.html) for the tool.

To access arbitrary angle rotations about the $x$, $y$, and $z$ axis of the Bloch sphere, we can use the `rx`, `ry`, and `rz` commands.

In the code block, we plot the effect of rx on the state $\ket{0}$ for a variety of angles.  You can see that when the angle is $\pi$, the rotation gate `rx` is equivalent to the `x` gate.  Feel free to edit the code bloch to apply the `ry` or `rz` gates to various angles on different initial states.  

In [ ]:
# set the number of angles to plot
num_angles = 15

@cudaq.kernel
def rotate_x(angle : float):
    qubit = cudaq.qubit()

    # Apply the unitary transformation
    # Rx(θ) = |  cos(θ/2)  -isin(θ/2) |
    #         | -isin(θ/2)  cos(θ/2)  |
    rx(angle, qubit)


blochSphere = qutip.Bloch()
for n in range(num_angles+1):
    theta = np.pi*n/num_angles
    sphere = cudaq.add_to_bloch_sphere(cudaq.get_state(rotate_x, theta), blochSphere)

# Display the Bloch spheres side by side in 2 rows and 2 columns
cudaq.show(blochSphere)




### Controlled gates

Earlier we examined the CNOT gate (`x.ctrl`) which is a controlled-x gate.  That is, the `x` gate is applied conditionally on the state of one or more of the control qubits.  Similarly, we can create controlled gates for any of our single qubit gates using the `.ctrl` suffix.  For instance `t.ctrl` will apply a `t` gate conditionally on the state of the control qubit.



With just a few of these gates, we can carry out some interesting computations.  Let's check out one step of Shor's algorithm in the next section.

## Application: modular arithmetic


Shor's algorithm is a well-known example that has the potential to break RSA encryption, assuming it could be executed on a fault-tolerant quantum computer at a sufficiently large scale. While we won’t be implementing the full algorithm here (you can explore more details [here starting on page 341](https://www.thomaswong.net/introduction-to-classical-and-quantum-computing-1e4p.pdf), [Chapter 4 of these lecture notes](https://arxiv.org/pdf/2311.08445)  or [chapter 3 of Mermin's book here](http://mermin.lassp.cornell.edu/qcomp/CS483.html) and you can view the code for Shor's algorithm [here](https://nvidia.github.io/cuda-quantum/latest/applications/python/shors.html)), we will focus on one step of the algorithm: performing modular arithmetic. This will serve as a demonstration of the concepts we've covered so far and offer an opportunity to encode classical information (in this case, an integer) onto qubits.
The circuit shown below, inspired by [this article](https://physlab.org/wp-content/uploads/2023/05/Shor_s_Algorithm_23100113_Fin.pdf), performs the operation of multiplying $5y$ modulo $21$, with the initial state representing various values of $y = 1, 4, 5, 16, 17, 20$.

![circuit for computing 5y mod 21 for certain values of y](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/modular_circuit.png?raw=1)

 For instance, if we want to calculate $5 \times 1 \mod 21$, we would encode $y = 1$ by preparing the quantum state $\ket{10000}$ through the application of an $X$ gate to qubit $q_0$:

![circuit for computing 5y mod 21 for certain values of y](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/modular_circuit_y_equals_1.png?raw=1)



### Exercise 6
Edit the code below to carry out the instructions of the circuit diagrams above to compute $5*y\mod{21}$ and verify that the outcome of the circuit is as expected.  For example, when the circuit is initialized as $y = 20$, we should expect to sample the binary representation of 16 (which is `00001`) since $5*20 = 100 \equiv 16\mod{21}$.



In [ ]:
# EXERCISE 6

import random

@cudaq.kernel
def modular_mult_5_21(qubits : cudaq.qvector):
    """"Kernel based off of the circuit diagram in
    https://physlab.org/wp-content/uploads/2023/05/Shor_s_Algorithm_23100113_Fin.pdf
    Modifications were made to change the ordering of the qubits.
    """
    # Edit code below this line


    # Edit code above this line

@cudaq.kernel
def encode_integer(qubits: cudaq.qvector, binary_rep: list[int]):
    """Kernel takes as input a list of qubits and the binary representation
    of an integer as a list. The kernel adds X-gates to the qubits to encode
    the binary list, placing an X gate on qubit i if there is a 1 in the ith location
    on the binary_rep list"""
    # Edit code below this line


    # Edit code above this line


def decimal_to_binary_list(number):
    # Check if the input number is valid (non-negative integer)
    if number < 0:
        raise ValueError("Number must be a non-negative integer.")

    # Convert the number to binary using bin() function and strip the '0b' prefix
    binary_string = bin(number)[2:]

    # Convert the binary string to a list of integers (0s and 1s)
    binary_list = [int(bit) for bit in binary_string]

    return binary_list

@cudaq.kernel
def mult_y_by_5_mod21(binary_list: list[int]):

    # Allocate qubits
    qubits = cudaq.qvector(5)

    # Initialize qubits in the state representing 1, 4, 5, 16, 17, or 20
    encode_integer(qubits, binary_list)
    # Apply the multiplication by 5 mod 21 kernel
    modular_mult_5_21(qubits)

values = [1,4,5,16, 17, 20]
number = random.choice(values) # selects a number from the list values randomly
# Convert number into a binary representation stored as a list of 0s and 1s
binary_list = decimal_to_binary_list(number)

results = cudaq.sample(mult_y_by_5_mod21, binary_list, shots_count = 200)
print("Multiplying {} by 5 mod 21 results in the bitstring {}".format(number,results.most_probable()))


### Exercise 7
Create a kernel that carries out modular exponentiation 5^x mod 21 using the <code>mult_y_by_5_mod21</code> of the previous exercise. Hint: $5^6 \mod(21) = 1$.  


In [ ]:
# EXERCISE 7

@cudaq.kernel
def modular_exp_kernel(exponent: int):
    """Kernel computes 5^x mod 21
    Parameters:
    -----------
    exponent : int
        the value x for the computation 5^x mod 21

    Returns:
    --------
    binary_rep : string
        binary representation for 5^x mod 21 as a string of 0s and 1s
    """
    # Allocate and intialize qubits
    qubits = cudaq.qvector(5)

    # Edit code below this line

    # Encode y = 1


    # Multiply y = 1 by 5 exp times


    # Edit code above this line


def modular_exp(exponent: int):
    sample_result = cudaq.sample(modular_exp_kernel, exponent, shots_count = 100).most_probable()
    return sample_result

for x in range(0,7):
    result = modular_exp(x)

    print("5^{} mod 21 in binary representation is {}".format(x, result))

# Add a Bit of Variation: Write your first variational program
$
\renewcommand{\ket}[1]{|{#1}\rangle}
\renewcommand{\bra}[1]{\langle{#1}|}
$

We begin with the same framework for developing a quantum program as used before, only in this lab we use parameterized gates.  Then, later, we adapt this framework into a hybrid quantum-classical workflow to identify parameters that optimize a cost function.  The adapted framework is outlined below:

1. Initialize parameters
2. Encode information into the quantum state by initializing qubit(s)
3. Manipulate the quantum state of the qubit(s) with quantum gate(s) using the  parameters
4. Extract information from the quantum state by measuring the state of the qubit(s) and computing the expectation value of the cost function
5. Run a classical optimizer to either determine a new state of parameters and repeat from step 2, or converge to an optimal solution.

These steps, of what is known as a **variational quantum algorithm**, are outlined in the diagram below.  


![image of a quantum circuit with the three partsL encode information, manipulate quantum states, extract information](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/variational-diagram.png?raw=1)


---
## Comparing Classical Random Walks and Discrete Time Quantum Walk (DTQW)

Quantum walks are a generalization of classical random walks. Unlike classical random walks, where probabilities dictate movement, quantum walks rely on amplitudes. Interference between paths leads to a faster spread over the position space, making quantum walks useful for tasks like quantum search algorithms ([Li and Sun](https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.6.033042) and [Wong](https://arxiv.org/pdf/2011.14533)) and for generating probability distributions as we'll explore here.

We begin this section with the definitions of classical random walks and discrete time quantum walks (DTQW). In Section 1.2, we use CUDA-Q to encode an example of a DTQW into a CUDA-Q kernel.

### Classical Random Walks

Let's consider a discrete walk taking place on a line. In this scenario, we envision a walker progressing along the
x-axis in discrete increments, with the movement governed by predefined rules based on coin flips.  A classical random walk can be viewed as a decision tree as in the diagram below.  In this scenario, a walker begins in the center of the line. After a coin flip, the walker moves to the left or right depending on the outcome of the coin flip (e.g., flipping a heads will send the walker one step to the left and flipping a tails will send the walk one step to the right).  After `num_time_steps` coin flips, we record the position of the walker.  We repeat this experiment multiple times and generate a histogram of the ending position of the walker in each experiment.

![classical walk](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/classical-walk.png?raw=1)


Let's see this in action with the random walk widget below. The green dot represents the walker whose movements are dictated by flips of a coin whose fairness is controlled by the probability slider. The ending position from each run of the experiment is recorded in the histogram.  Experiment by changing the number of total steps and the probabilities. What patterns do you notice?

> **Note:** If the widget does not appear below, you can access it directly using [this link](https://nvidia.github.io/cuda-q-academic/quantum-applications-to-finance/images/random_walk.html).

<iframe src="https://nvidia.github.io/cuda-q-academic/quantum-applications-to-finance/images/random_walk.html" width="800" height="600"></iframe>

### Discrete Time Quantum Walks

A **discrete-time quantum walk (DTQW)** is the quantum analogue of a classical random walk. In this scenario, both the position of the walker and the coin are represented as quantum states, $\ket{\psi_W}$ and $\ket{\psi_C}$, respectively. Throughout this walk, the state of the walker changes according to a quantum shift operators ($S_+, S_-$) which depend on the state of the coin $\ket{\psi_C}$. In particular, the $S_{-}$ operation checks if $\ket{\psi_C}$ in the state $\ket{0}$ (i.e., heads) and if so, induces a shift left operation on the quantum walker. Similarly, $S_{+}$ is defined for tails and shifting right. At each time step, the state of the coin changes (i.e., is "flipped") via a quantum operation ($F$), which we'll call the coin-flip operator. Unlike the classical random walk, we will not know the position of the walker until the end of the process, when the quantum state of the walker is measured, and we'll never know the state of the coin!

In the diagram below we illustrate a 2-step quantum walk where the coin-flip operator $F$ is the Hadamard gate.  Notice how the final distribution of the walker's possible positions differs from the classical case. In particular, the state of the walker after 2 time steps is in superposition.  Moreover, upon measurement of the final state, the walker never ends up in the location $4 = |0100\rangle$, as might occur with two tails in the classical case depicted above.


![dtqw-superposition-coin.png](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/dtqw-superposition-coin.png?raw=1)


#### The Line that the Walker Traverses
 For our example, let's consider a line that contains $16$ positions, labeled $0,1,\cdots,15$.  We can use $4$ qubits and the computational basis states, ($\ket{\bf{0}} = \ket{0000}, \ket{\bf{1}} = \ket{0001}, \ket{\bf{2}} = \ket{0010}, \cdots, \ket{\bf{15}} = \ket{1111}$), to represent positions on the line.  In other words, the computational basis state $\ket{\bf{i}}$ represents the $i^{th}$ location on the line.


#### The Walker's Position State
The walker's position after $t$ time steps is given by a linear combination of the computational basis states $$\ket{\psi_W(t)} = \sum_{i}\alpha_i(t)\ket{\bf{i}},$$ for some $\alpha_i(t)\in\mathbb{C}$ satisfying $\sum_{i}|\alpha_i(t)|^2 = 1$.  For instance, if the walker began the experiment at position $0$, then the state of the walker at time $0$ would be $\ket{\psi_W(0)}=\ket{0000}$.  More interestingly, the walker might begin the experiment in a superposition of two or more positions.  For example, the initial state of the walker could be the positions $2$ and $3$.  In this case the walker's initial state is $\ket{\psi_W(0)}=\frac{1}{\sqrt{2}}(\ket{\bf{2}}+\ket{\bf{3}}) = \frac{1}{\sqrt{2}}\ket{0010}+\frac{1}{\sqrt{2}}\ket{0011}$ as depicted in the image below.

![dtqw-superposition-walker.png](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/dtqw-superposition-walker.png?raw=1)

You can explore more steps in a quantum walk with the visualization tool below.


---
## Programming a variational DTQW with CUDA-Q

Let's start coding up the variational DTQW in CUDA-Q.  This is depicted in the diagram below.

![dtqw-no-optimization.png](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/dtqw-no-optimization.png?raw=1)

We'll explain each portion of this diagram as we work through this section.
First, let's start with the purple optional quantum walker initialization in the upper left hand corner of the diagram.

### Exercise  8:
Edit the code block below to create a kernel that generates the initial state $\ket{\psi_W(0)}=\frac{1}{\sqrt{2}}\ket{0000}+\frac{1}{\sqrt{2}}\ket{1111}$ using an <code>x</code> gate and CNOT gates <code>x.ctrl(control_qubit, target_qubit)</code>.
    


In [ ]:
# EXERCISE 8

@cudaq.kernel
def initial_state(qubits : cudaq.qview):
    """ Apply gates to the qubits to prepare the GHZ state
    Parameters
        qubits: cudaq.qvector
        qubits for the walker
    """

    # Edit the code below this line




    # Edit the code above this line

Let's verify that our solution is correct by using the `cudaq.get_state` and _`.amplitudes` commands.

In [ ]:
# Create a quantum kernel that initializes the state of the walker

num_qubits = 4

@cudaq.kernel
def walker(num_qubits : int):
    """ Kernel to initialize the state of the walker
    Parameters
        num_qubits : int
        Number of qubits for the walker
    """
    # Initialize the qubits
    qubits = cudaq.qvector(num_qubits)

    # Apply the initial state to the qubits
    initial_state(qubits)

# Get the state of the walker after applying the quantum kernel
state = cudaq.get_state(walker, 4)

# Return the amplitudes of |0000> and |1111>
state_amplitudes = state.amplitudes([[0,0,0,0], [1,1,1,1]])

# Print
precision = 4
print('Walker state array of coefficients:', np.round(np.array(state), precision))
print('Walker statevector: {} |0000> + {} |1111>'.format(np.round(state_amplitudes[0],precision), np.round(state_amplitudes[1],precision)))

#### Walking the Line

The next step (no pun intended) is to code incrementer and decrementer operations, drawn as light green and yellow blocks and denoted by $INC$ and $DEC$ in the diagram.  These will change the state of the walker with shifts to the left and right on the number line.  We'll use these operations to build up the shift operators $S_-$ and $S_+$ which depend on the state of the coin, depicted in the diagram with the control operators.  

First let's define $INC$  so that when applied to a basis state $\ket{x}$, the result is $\ket{(x+1)_{\mod{16}}}$ for $x\in \{0,\cdots 15\}$. The kernel for INC is defined below.  


In [ ]:
# Define a kernel on 4 qubits for the INC operation that
# maps |x> to |x+1> mod 16 and verify that it works as expected for |0000>

@cudaq.kernel
def INC(qubits : cudaq.qview):
    x.ctrl([qubits[3], qubits[2], qubits[1]], qubits[0])
    x.ctrl([qubits[3], qubits[2]], qubits[1])
    x.ctrl(qubits[3], qubits[2])
    x(qubits[3])


# Create a kernel that applies the INC to the 4 qubit state |0000>
@cudaq.kernel
def check_incrementer_kernel():
    qubits = cudaq.qvector(4)
    INC(qubits)

# Print the INC circuit
print(cudaq.draw(check_incrementer_kernel))

result = cudaq.sample(check_incrementer_kernel).most_probable()
print('Incrementer kernel |0000> -> |{}>'.format(result))

### Exercise  9:

Using the fact that DEC is the inverse of INC, create the DEC kernel by completing the code block below.




In [ ]:
# EXERCISE 9

# Define a kernel on 4 qubits for the DEC operation that
# maps |x> to |x-1> mod 16 and verify that it works as expected for |0001>

@cudaq.kernel
def DEC(qubits : cudaq.qview):
    # Edit the code below this line




    # Edit the code above this line

# Create a kernel that applies the DEC to the 4 qubit state |0000>
@cudaq.kernel
def check_decrementer_kernel():
    qubits = cudaq.qvector(4)
    # Initialize the qubits to |0001>
    x(qubits[3])
    DEC(qubits)

result = cudaq.sample(check_decrementer_kernel).most_probable()
print('Decrementer kernel |0001> -> |{}>'.format(result))

#### The State of the Coin

Not only is the position of the walker at any given time step a linear combination of the computational basis states, but also the coin is a linear combinations of heads:

$$\ket{\uparrow} \equiv \ket{0} = \begin{pmatrix}1 \\ 0 \end{pmatrix}$$

and tails:

$$\ket{\downarrow} \equiv \ket{1} = \begin{pmatrix}0 \\ 1 \end{pmatrix}$$

and depends on $t$: $$\ket{\psi_C(t)} = \beta_0(t)\ket{0}+\beta_1(t)\ket{1},$$ for $\beta_0,\beta_1\in\mathbb{C}$ with $|\beta_0(t)|^2 + |\beta_1(t)|^2 = 1.$  In case it helps to think of this qubit as "heads up" and "heads down" and distinguish it from the position state, we introduced the alternative (and optional) notation $\ket{\uparrow}$ and $\ket{\downarrow}$ for the zero-state (heads up) and the one-state (heads down), respectively.

Let's look at a couple of examples. We might begin the experiment with a coin in the zero (heads) state: $\ket{\psi_C(0)}=\ket{0}$.  Another option would be to start the experiment with the coin in a state of superposition of both heads and tails: $\ket{\psi_W(0)}=\frac{1}{\sqrt{2}}\ket{0}+\frac{1}{\sqrt{2}}\ket{1}\equiv\ket{+}$.

#### Changing the State of the Coin

The flipping of the coin is carried out by a quantum operation, $F$.  If the coinflip operation is the bitflip operation, $X$,
and the initial state of the coin is $\ket{\psi_C(0)} = \ket{0}$, then, flipping the coin would just change the state from $\ket{0}$ to $\ket{1}$ &mdash; much like the classical random walk.   

A common and more interesting quantum coinflip operation is the Hadamard operator, $F=H$, as illustrated in the diagrams of the DTQW that appeared earlier in this notebook.
Applying this operator iteratively to the coin in the initial state $\ket{\psi_C(0)} = \ket{0}$, will result in alternating coin states $\ket{+}$ and $\ket{0}$.  The initializing the state of the coin is depicted in the diagram as a purple block on the lower left corner.




We're not limited to constant coin operators, and could consider a coin operator that depends on parameters such as the Grover operator used for searching a graph for marked vertices ([Li and Sun](https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.6.033042) and [Wong](https://arxiv.org/pdf/2011.14533)). Moreover, the coin operator might depend on parameters that change from one step to the next and can be learnable ([Chang et al., 2023](https://arxiv.org/pdf/2302.12500) [Chang et al., 2025](https://arxiv.org/pdf/2504.13532)), as we'll see in the next example.

So far we have considered single-qubit operations that rotate the Bloch sphere by a fixed amount.  For instance the `x`-gate corresponds to a rotation of the Bloch sphere about the $x$-axis by 180 degrees, and the `t`-gate rotates a statevector 45 degrees about the $Z$-axis.  What about all the other rotation angles and all the other axis of rotation?  The `u3`-gate is a universal rotation gate that allows us to define arbitrary rotation gates.  The unitary matrix for this gate is:

$$ U_3(\theta, \varphi,\lambda) = \begin{pmatrix} \cos{\frac{\theta}{2}} &  -e^{i\lambda}\sin{\frac{\theta}{2}} \\  
                         e^{(i\varphi} \sin{\frac{\theta}{2}} &    e^{i(\lambda + \varphi)} \cos{\frac{\theta}{2}} \end{pmatrix},$$

where $\theta$, $\varphi$, and $\lambda$ are variables which we will assign values to before executing the kernel. We've drawn this as a dark green block in the diagram.

Experiment with different gates and different values of $\theta$, $\varphi$, and $\lambda$ in the code block below to see the effect of a coin flip defined by the `u3`-gate on a coin which is initiated in the heads up, zero-state.

In [ ]:
# Coin flip with the u3-gate applied to a coin in the zero-state

# theta = np.pi/2 and phi = Lambda = 0 is the Hadamard gate
theta = np.pi/2     #CHANGE ME
phi = 0             #CHANGE ME
Lambda = 0          #CHANGE ME

# Parameterized kernel for the coin flip
@cudaq.kernel
def coin_flip_with_u(theta : float, phi : float, Lambda: float):

    # Initialize the coin state
    coin_qubit = cudaq.qubit()

    # Apply a coin flip with the u3-gate and given parameters
    u3(theta, phi, Lambda, coin_qubit)



# Get the state of the coin after the flip
coin_flip_result = cudaq.get_state(coin_flip_with_u, theta, phi, Lambda)

cudaq.show(cudaq.add_to_bloch_sphere(coin_flip_result))



#### Changing the State of the Walker Based on Coin Flips:



### Exercise 10
Now we're ready to put all of these kernels together to program one step of the DTQW.  Let's set the initial state of the walker to be $\ket{\psi_W(0)} = \ket{0101}$. The steps $S_+$ and $S_-$, which depend on the state of the coin, can be modeled with controlled-gates and the INC and DEC operations, respectively.  We've defined the $S_+$ step below, which uses a controlled-gate to apply INC to the walker state, if the coin is in the one-state $\ket{\uparrow}$.  How would you adapt the code to call up the $DEC$ operator, when the coin qubit is in the $\ket{0}$ state?






In [ ]:
# EXERCISE 10
# Fill in the code to carry out the S- step

# Pick your favorite values
theta = np.pi/4     #CHANGE ME
phi = np.pi/4       #CHANGE ME
lam = 0.0          #CHANGE ME

# Set the number of qubits
num_qubits = 4

@cudaq.kernel
def DTQW_one_step(num_qubits: int, theta : float, phi : float, lam : float):
    walker_qubits = cudaq.qvector(num_qubits)
    coin_qubit = cudaq.qvector(1)
    # Initial walker state |0101>
    x(walker_qubits[1])
    x(walker_qubits[3])

    # Initial coin state
    #h(coin_qubit[0]) #Uncomment this line to start with a superposition of heads and tails instead of |0>

    # One quantum walk step
    # Coin operation F=u3
    u3(theta, phi, lam, coin_qubit)

    # Walker's position change
    # Shift right (S+) when the coin is |1>
    cudaq.control(INC, coin_qubit[0], walker_qubits)

    # Shift left (S-) when the coin is |0>
    # EDIT CODE BELOW THIS LINE



    # EDIT CODE ABOVE THIS LINE

    mz(walker_qubits)

# Visualize the kernel for the quantum walk
print(cudaq.draw(DTQW_one_step, num_qubits, theta, phi, lam))

# Sample the kernel for the quantum walk
result = cudaq.sample(DTQW_one_step, num_qubits, theta, phi, lam, shots_count=1000)
print(result)

The code block below helps us to visualize the results of the sampling:

In [ ]:
# Define a function to draw the histogram of the results

def plot_results(result, num_qubits):
    # Define a dictionary of results

    # Initialize the dictionary with all possible bit strings of length 4 for the x axis
    result_dictionary = {}

    # Generate all possible bit strings of length num_qubits
    for i in range(2**num_qubits):
        bitstr = bin(i)[2:].zfill(num_qubits)
        result_dictionary[bitstr] = 0

    # Update the results dictionary of results from the circuit sampling
    for k,v in result.items():
        result_dictionary[k] = v

    # Convert the dictionary to lists for x and y values
    x = list(result_dictionary.keys())
    y = list(result_dictionary.values())

    # Create the histogram
    plt.bar(x, y, color='#76B900')

    # Add title and labels
    plt.title("Sampling of the DTQW")
    plt.xlabel("Positions")
    plt.ylabel("Frequency")

    # Rotate x-axis labels for readability
    plt.xticks(rotation=45)

    # Show the plot
    plt.tight_layout()
    plt.show()

plot_results(result, num_qubits)

Go ahead and experiment with the single DTQW code above by changing the parameter values and initial position to get a sense of how the $S_+$ and $S_-$ operators work.

### Exercise 11
Your next task is to edit the code below to perform <code>num_time_steps</code> iterations of the quantum walk, sample the outcomes, and plot a histogram of the final position state. Keep in mind that you cannot embed DTQW_one_step within a loop, as the DTQW algorithm prescribes taking a single measurement only after completing all steps, rather than after each individual step.
    



In [ ]:
# EXERCISE 11

# Set a variable for the number of time steps
num_time_steps = 4 # CHANGE ME to see the effect of different length walks

# Pick your favorite values
theta = np.pi/4     #CHANGE ME
phi = np.pi/4       #CHANGE ME
lam = 0.0          #CHANGE ME

# Set the number of qubits
num_qubits = 4

# EDIT CODE BELOW THIS LINE

@cudaq.kernel()
def DTQW_multi_step(num_qubits: int, theta : float, phi : float, lam : float, num_time_steps : int):
    walker_qubits = cudaq.qvector(num_qubits)
    coin_qubit = cudaq.qvector(1)
    # Initial walker state |0101>
    x(walker_qubits[1])
    x(walker_qubits[3])

    # Initial coin state
    #h(coin_qubit[0]) #Uncomment this line to start with a superposition of heads and tails instead of |0>

    # Flip the coin num_time_steps and shift the walker accordingly




    # Measure the state of the walker
    mz(walker_qubits)

# EDIT CODE ABOVE THIS LINE

# Sample the kernel for the quantum walk
result_multi_step = cudaq.sample(DTQW_multi_step, num_qubits, theta, phi, lam, num_time_steps, shots_count=1000)
print(result_multi_step)

# Draw the histogram of the results after one step
plot_results(result_multi_step, num_qubits)

Change the initial walker's position and parameter values for the coin flip operator to see the variety of distributions that can be generated with a quantum random walk.  

Our next goal will be to use the quantum random walk to load a probability distribution into a quantum state.  This will require us to be able to match a given probability distribution with parameter values that will generate that distribution.  To do this, we'll use a classical optimizer.  That's the challenge of the next notebook.

# Converge on a Solution: Write your first hybrid variational program
$
\renewcommand{\ket}[1]{|{#1}\rangle}
\renewcommand{\bra}[1]{\langle{#1}|}
$

In this part, we'll compute expectation values and use a classical optimizer to identify parameters to optimize a cost function, completing the coding of the diagram below:

![image of a variational discrete time quantum walk](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/dtqw-diagram.png?raw=1)

In [ ]:
# Define a kernel on 4 qubits for the INC operation that
# maps |x> to |x+1> mod 16

@cudaq.kernel
def INC(qubits : cudaq.qview):
    x.ctrl([qubits[3], qubits[2], qubits[1]], qubits[0])
    x.ctrl([qubits[3], qubits[2]], qubits[1])
    x.ctrl(qubits[3], qubits[2])
    x(qubits[3])


# Define a kernel on 4 qubits for the DEC operation that
# maps |x> to |x-1> mod 16

@cudaq.kernel
def DEC(qubits : cudaq.qview):
    x(qubits[3])
    x.ctrl(qubits[3], qubits[2])
    x.ctrl([qubits[3], qubits[2]], qubits[1])
    x.ctrl([qubits[3], qubits[2], qubits[1]], qubits[0])


---
## Defining Hamiltonians and Computing Expectation Values

A significant challenge in quantum computing is the efficient encoding of classical data into quantum states that can be processed by quantum hardware or simulated on classical computers. This is particularly crucial for many financial applications, where the first step of a quantum algorithm often involves efficiently loading a probability distribution. For instance, to enable lenders to price loans more accurately based on each borrower's unique risk profile, it is essential to estimate individual loan risk distributions (such as default and prepayment) while accounting for uncertainties in modeling and macroeconomic factors. Efficiently implementing this process on a quantum computer requires the ability to load a log-normal or other complex distribution into a quantum state [(Breeden and Leonova)](https://www.tandfonline.com/doi/full/10.1080/01605682.2022.2115415).

Financial markets exhibit complex probability distributions and multi-agent interactions that classical models struggle to capture efficiently. Quantum walks can be used to generate probability distributions of market data. The quantum walk approach offers:
* Flexible modeling of price movements
* Better representation of extreme events compared to classical methods
* Ability to capture asymmetric return distributions [(Backer et al)](https://arxiv.org/pdf/2403.19502).

### The Problem

This tutorial explores how to load a probability distribution with a given property (in this case, a fixed mean) into a quantum state. By following this example, you will gain the essential knowledge needed to comprehend more sophisticated algorithms, such as the multi-split-step quantum walks (mSSQW) method, as described by  [Chang et al.](https://arxiv.org/pdf/2302.12500) The mSSQW technique can be employed to load a log-normal distribution, which is useful for modeling the spot price of a financial asset at maturity.

**Pedagogical Remark:** The reason why we chose to examine the coarser problem of generating a distribution with a targeted mean as opposed to generating a targeted distribution itself (as in [Chang et al.](https://arxiv.org/pdf/2302.12500)) is that by considering the mean of the distribution we can introduce the concept of expectation value of a Hamiltonian which is central to many variational algorithm applications in chemistry, optimization, and machine learning.  

Let's begin by examining the final step in the quantum program template, which involves taking measurements and interpreting the results:

![image of a quantum circuit with the three parts: encode information, manipulate quantum states, extract information](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/circuit.png?raw=1)

Up to this point, for this step of the quantum program, we utilized `cudaq.get_state` and `cudaq.sample` to read out a statevector from the quantum kernel. However, this approach is not always practical, especially considering that the dimension of the statevector scales exponentially with the number of qubits. For example, describing the statevector for $100$ qubits would require storing $2^{100}$ amplitudes, which is extremely memory-intensive. In many quantum algorithms, we do not need a complete description of the statevector; instead, partial information like an "average" value often suffices.  This is where the concept of expectation value comes in.



### Illustrative Example of Expectation Value

Before we delve into our DTQW example, let's make this idea of "average" value of a quantum state more explicit by considering a specific example.

Take the quantum state $\ket{\psi} = \sum_j\alpha_j\ket{j}$, where $\alpha_j = \frac{1}{4}$ for all $j\in\{0, \cdots 15\}$. The probability amplitudes for the computational basis states of this example are graphed below:  


![equal-superposition.png](https://github.com/osbama/KBM608/blob/main/hands-on/hands-on-2-images/equal-superposition.png?raw=1)


#### Expectation Values



We can think of this as a probability distribution of a random variable and compute the **expectation value** (i.e., average or mean).  Graphically, we can determine that the expectation value is the position halfway between $\ket{7}$ and $\ket{8}$ (i.e., between the states $\ket{0111}$ and $\ket{1000}$). Analytically, we'd find this by computing $\sum^{15}_{j=0} p_j i$, where $p_j$ is the probability of measuring the state $\ket{j}$.  In this example we'd get $\sum^{15}_{j=0} p_j j = \frac{1}{16}\sum_{j=0}^{15}j = 7.5.$

Let's look into how, in general, we can deduce the expectation value $\sum^{15}_{j=0} p_j j$  from a state $\ket{\phi} = \sum_j\alpha_j\ket{j}$. We know that $|\alpha_j|^2 = p_j$, by definition of the probability amplitudes.  Therefore, to complete the computation, we need an operation to translate from $\ket{j}$ to $j$.  

This is where the Hamiltonian comes in.  We won't formally define a Hamiltonian in its full generality here (you can read more about Hamiltonians [in Scott Aaronson's lecture notes](https://www.scottaaronson.com/qclec/25.pdf)).  For this tutorial, we will consider a Hamiltonian to be a $16\times 16$ matrix that operates on a $4$-qubit quantum state through matrix multiplication. This matrix has certain additional properties, which we will not focus on for now.  For the purposes of this tutorial, all you need to know is that $H$ has the properties:
* $H(\ket{j}) = j\ket{j}$ for $j \in \{0,1,\cdots, 15\}$
* $H$ is linear, that is $H(c\ket{\psi}+d\ket{\phi}) = cH(\ket{\psi})+dH(\ket{\phi})$ for $c,d\in\mathbb{C}$ and states $\ket{\psi}$ and $\ket{\phi}$.

We've left the explicit formula for $H$ in the optional box below.  But assuming that we have such an operation, then we can define and denote the **expectation value** of this Hamiltonian with respect to a state $\phi$ as
$$ \bra{\phi}H\ket{\phi} \equiv \ket{\phi}^\dagger H(\ket{\phi}),$$
where the notation $\bra{\phi}$ is read as "bra phi" and comes from the bra-ket notation. It represents the conjugate transpose of the ket, $\ket{\phi}$. In other words, bra-$\phi$ ($\bra{\phi}$) is equal to $\ket{\phi}^\dagger$. Placing $\bra{\phi}$ next to $H(\ket{\phi})$ in the equation above represents the action of taking the dot product of  $\bra{\phi}$ with $H(\ket{\phi})$.

Let's compute $$ \bra{\psi}H\ket{\psi},$$ for the equal superposition state $\ket{\psi} = \sum_j\frac{1}{4}\ket{j} = \frac{1}{4}\begin{pmatrix}1 \\ 1\\ \vdots \\ 1 \end{pmatrix},$ graphed above.  By our choice of $H$ and its linearity, we get that

$$H\ket{\psi} = H(\sum_{j=0}^{15}\frac{1}{4}\ket{j}) = \frac{1}{4}\sum_{j=0}^{15}H(\ket{j}) = \frac{1}{4}\sum_{j=0}^{15}j(\ket{j})= \frac{1}{4}\begin{pmatrix} 0 \\ 1 \\ 2 \\ \vdots \\ 15 \end{pmatrix}.$$

Next combining this with $\bra{\psi}$ via a dot product, we get

$$ \bra{\psi}H\ket{\psi} = \frac{1}{4}\begin{pmatrix}1 & 1& \cdots & 1\end{pmatrix} \frac{1}{4}\begin{pmatrix} 0 \\ 1 \\ 2 \\ \vdots \\ 15 \end{pmatrix} =   \frac{1}{16}\begin{pmatrix}1 & 1& \cdots & 1\end{pmatrix}\begin{pmatrix} 0 \\ 1 \\ 2 \\ \vdots \\ 15 \end{pmatrix} = \frac{1}{16}(0+1+2+\cdots + 15) = \frac{1}{16}\sum_{j=0}^{15} j = 7.5, $$

which is exactly the expectation value that we found analytically at the beginning of this section.

> **Optional:**   The particular Hamiltonian that we need is the one that has the property $H\ket{j} = j\ket{j}$. In other words, the computational basis states are eigenvalues of the Hamiltonian and the eigenvalue of $\ket{j}$ is $j$.  The Hamiltonian that has this property is
> $$H = \sum_{j=0}^{15}j\ket{j}\bra{j},$$
>where $\ket{j}\bra{j}$ is the matrix product of $\ket{j}$ with $\bra{j}$.
> A more useful formulation of $H$ is
> $$ H =\sum_{j=0}^{15}j\ket{j}\bra{j} = 4(I-Z_0) + 2(I-Z_1)  + (I-Z_2) + \frac{1}{2}(I-Z_3) = 7.5I- (4Z_0 + 2 Z_1 + Z_2 +\frac{1}{2}Z_3),$$
> where $I$ is the identity matrix and $Z_j$ is the $16\times16$-dimensional matrix which applies the $Z = \begin{pmatrix} 1 & 0 \\ 0 & -1\end{pmatrix}$ to the $j^{th}$ qubit and holds all other qubits constant.  For example applying $Z_2$ to the state $\ket{1111}$ results in $-\ket{1111}$ and $Z_2$ applied to $\ket{1011}$ is $\ket{1011}$.  With some matrix algebra, you can verify the following equality holds for $j\in \{0,1,\cdots, 15\}$:
> $$ H\ket{j} = 7.5I\ket{j}- 4Z_0\ket{j} - 2 Z_1\ket{j} - Z_2\ket{j} -\frac{1}{2}Z_3\ket{j} = j\ket{j}.$$

#### Computing Expectation Values with `observe`
The `observe` function is used to compute expectation values of Hamiltonians.  Suppose that the state $\ket{\psi}$ is defined by a `cudaq.kernel` called `my_kernel`, and $H$ is a Hamiltonian stored as a `cudaq.operator`, named `my_hamiltonian`. We can compute the expectation value $\bra{\psi}H\ket{\psi}$ using `observe(my_kernel, my_hamiltonian).expectation()`.  For `observe` to work, the kernel shouldn't contain any measurements of the state of interest &mdash; otherwise, the state of the kernel would have collapsed to one of the basis states.  We can store $H$ as a `cudaq.operator` using `cudaq.spin.z` and `cudaq.spin.i` along with `+` and scalar multiplication `*`.  Let's walk through computing the expectation value for a probability distribution represented by a quantum state in the code block below.

In [ ]:
# Computing expectation value of an equal superposition state

# Set the number of qubits
num_qubits = 4

# Define a kernel for the state |psi> = 1/4(|0000> + |0001> + ...+ |1111>)
@cudaq.kernel
def equal_superposition(num_qubits : int):
    """ Apply gates to the qubits to prepare the GHZ state
    Parameters
        num_qubits : int Number of qubits
    """
    # Edit the code below this line
    qubits = cudaq.qvector(num_qubits)
    for i in range(len(qubits)):
        h(qubits[i])

# Define the Hamiltonian
position_hamiltonian =  7.5*cudaq.spin.i(0) - 4*cudaq.spin.z(0)-2*cudaq.spin.z(1)-cudaq.spin.z(2)-0.5*cudaq.spin.z(3)

# Compute the expectation value
expectation_value = cudaq.observe(equal_superposition, position_hamiltonian, num_qubits).expectation()

print(expectation_value)

As anticipated, the expectation value of the equal superposition state is computed to be 7.5.  In this section we introduced ideas that are essential for many other applications of variational quantum algorithms.  In particular, we introduced the Hamiltonian $H$ for computing the average value of a quantum state if we identify the computational basis states with integers.  We also demonstrated how to compute the expectation value of the Hamiltonian applied to a quantum state  $\bra{\psi}H\ket{\psi}$ using the `observe` command.  In the next section, we'll apply all of this to our DTQW problem.  

### Expectation value of a DTQW

We are now prepared to apply what we've learned to the problem of finding a Discrete-Time Quantum Walk (DTQW) that yields a probability distribution with a specific mean. Let's aim to generate a distribution from a quantum walk with a mean of $3$. In the code block below, you'll find a parameterized kernel for the DTQW based on the code that we developed in Lab 3.  We've just added a flag so that we both sample and compute expectation values with the kernel.

Do your best to adjust the parameter values to achieve a state with an average close to $3$. It's challenging, isn't it? There are just so many combinations of parameter values to experiment with!

In [ ]:
# Set a variable for the number of time steps
num_time_steps = 6

# Pick your favorite values
theta = np.pi/2    #CHANGE ME
phi = np.pi/2     #CHANGE ME
lam = 0.0        #CHANGE ME

# Set the number of position qubits
num_qubits = 4

@cudaq.kernel()
def measure(qubits : cudaq.qview):
    mz(qubits)

@cudaq.kernel()
def DTQW_for_expectation_value_computation(num_qubits: int, parameters : list[float], num_time_steps : int, with_measurement: int):
    walker_qubits = cudaq.qvector(num_qubits)
    coin_qubit = cudaq.qvector(1)

    theta = parameters[0]
    phi = parameters[1]
    lam = parameters[2]

    # Initial walker state |9> = |1001> for possibly faster convergence
    #x(walker_qubits[3])
    #x(walker_qubits[2])

    # initial coin state
    h(coin_qubit[0])

    #for i in range(1, num_qubits):
    #    x.ctrl(walker_qubits[0], walker_qubits[i])

    # Flip the coin num_time_steps and shift the walker accordingly
    for _ in range(num_time_steps):

        # One quantum walk step
        # Coin operation F=u3
        u3(theta, phi, lam, coin_qubit)

        # Walker's position change
        # Shift right (S+) when the coin is |1>
        cudaq.control(INC, coin_qubit[0], walker_qubits)

        # Shift left (S-) when the coin is |0>

        x(coin_qubit[0])
        cudaq.control(DEC, coin_qubit[0], walker_qubits)
        x(coin_qubit[0])

    if (with_measurement==1):
        # Measure walker qubits
        measure(walker_qubits)


# Sample the kernel for the quantum walk
result_aiming_for_mean_of_3 = cudaq.sample(DTQW_for_expectation_value_computation, num_qubits, [theta, phi, lam], num_time_steps, 1, shots_count=10000)
print('sampling results with the coin qubit:', result_aiming_for_mean_of_3)


# Define a function to draw the histogram of the results ignoring the coin qubit

def plot_results_without_coin_qubit(result, num_qubits):
    # Initialize the dictionary with all possible bit strings of length 4 for the x axis
    result_dictionary = {}

    # Generate all possible bit strings of length num_qubits
    for i in range(2**num_qubits):
        bitstr = bin(i)[2:].zfill(num_qubits)
        result_dictionary[bitstr] = 0

    # Update the results dictionary of results from the circuit sampling
    for k,v in result.items():
        result_dictionary[k] = v

    # Convert the dictionary to lists for x and y values
    x = list(result_dictionary.keys())
    y = list(result_dictionary.values())

    # Create the histogram
    plt.bar(x, y, color='#76B900')

    # Add title and labels
    plt.title("Sampling of the DTQW")
    plt.xlabel("Positions")
    plt.ylabel("Frequency")

    # Rotate x-axis labels for readability
    plt.xticks(rotation=45)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Draw the histogram of the results after one step
plot_results_without_coin_qubit(result_aiming_for_mean_of_3, num_qubits)

# Compute the expectation value
exp_value = cudaq.observe(DTQW_for_expectation_value_computation, position_hamiltonian, num_qubits, [theta, phi, lam], num_time_steps,0).expectation()
print('The expectation value <ψ|H|ψ>  using parameters ({:.6f}, {:.6f}, and {:.6f}), is {} '.format(theta, phi, lam, exp_value))

---
## Identify Parameters to Generate a Targeted Mean Value


  

The code block below defines the cost function and creates a list to record its values at each iteration of the loop.

In [ ]:
# Create a list to store the cost values from each iteration of the variational algorithm
cost_values = []

# Compute the cost for a given set of parameters
def cost(parameters):
    """Returns the MSE between the targeted mean of 3 and the expectation value of the DTQW as our cost,
    which we want to minimize. The cost for the given parameters is stored in the cost_values list.
    Parameters
        parameters : list[float] The parameters to be optimized
    Returns
        cost_val : float The cost value
    """
    # Compute the expectation value
    expectation_value = cudaq.observe(DTQW_for_expectation_value_computation, position_hamiltonian, num_qubits, parameters, num_time_steps, 0, shots_count = 50000).expectation()
    # Compute the cost value
    cost_val = np.sqrt((expectation_value-3)**2)
    cost_values.append(cost_val)
    return cost_val

Below we use our built-in optimization suite (`cudaq.optimizers`) to minimize the cost function. Specifically, we will select the gradient-free Nelder Mead classical optimization algorithm.

In [ ]:
# Define a CUDA-Q optimizer
optimizer = cudaq.optimizers.NelderMead()

# Set initial parameter values
optimizer.initial_parameters = [theta, phi, lam]

# Optimize the cost function
# The optimizer will try to minimize the cost function defined above
result = optimizer.optimize(dimensions=3, function=cost) #dimensions is the number of parameter values

print('The minimal cost found is ',result[0])
print('The optimized parameters are ',result[1])

Let's plot the cost values for each iteration of the variational algorithm to see the convergence.

In [ ]:

# Plotting how the value of the cost function decreases during the minimization procedure.
plt.title("Convergence of the Variational Quantum Algorithm")
x_values = list(range(len(cost_values)))
y_values = cost_values

plt.plot(x_values, y_values)

plt.xlabel("Iterations")
plt.ylabel("Cost Value")

Finally, let's use the optimal parameters to carry out the DTQW to verify that we have generated a probability distribution with a mean close to 3.

In [ ]:
optimal_parameters = result[1]
result_optimized = cudaq.sample(DTQW_for_expectation_value_computation, num_qubits, optimal_parameters, num_time_steps, 1, shots_count=10000)
print('sampling results with the coin qubit:', result_optimized)

# Draw the histogram of the results after one step
plot_results_without_coin_qubit(result_optimized, num_qubits)

# Compute the expectation value
exp_value = cudaq.observe(DTQW_for_expectation_value_computation, position_hamiltonian, num_qubits, optimal_parameters, num_time_steps,0).expectation()
print('The expectation value  <ψ|H|ψ>  is ',exp_value)